In [54]:
import xml.etree.ElementTree as ET
import pandas as pd

In [58]:
def read_xml(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    temp = []
    for child in root:
        # child[0] - speech; child[1] - evaluation; child[2] - url
        temp.append([child[0].text.strip(), child[1].text.strip(), child[2].text.strip()])
    df = pd.DataFrame(temp)
    df.columns = ['speech', 'evaluation', 'url']
    df = df[df.evaluation.isin(['0', '+', '-'])]
    df = df.reset_index().drop('index', axis=1)
    return df

In [59]:
train = read_xml('data/train/news_eval_train.xml')
test = read_xml('data/test/news_eval_test.xml')

In [63]:
train['part'] = 'train'
test['part'] = 'test'

In [67]:
combined = pd.concat([train, test]).reset_index().drop('index', axis=1)

In [72]:
combined = pd.concat([combined, pd.get_dummies(combined.evaluation)], axis=1)

In [74]:
combined.columns = ['speech', 'evaluation', 'url', 'part', 'positive', 'negative', 'neutral']

In [75]:
combined.head()

,speech,evaluation,url,part,positive,negative,neutral
0,"Далее в своей проповеди он напомнил, что по би...",0,http://www.blagovest-info.ru/index.php?ss=2&am...,train,0,0,1
1,"""Меня отпустили. У Коли @nlyaskin забирают вещ...",-,http://asiareport.ru/index.php/news/14440-chir...,train,0,1,0
2,"В интервью РИА ""Новости"" уполномоченный по пра...",0,http://www.rosbalt.ru/federal/2012/04/08/96718...,train,0,0,1
3,Бывший главный тренер сборной Англии Грэм Тэйл...,0,http://www.sports.ru/football/139754406.html,train,0,0,1
4,"""На телах жертв были обнаружены многочисленные...",-,http://moldinfo.ru/narod/2939-jitel-vengerskoy...,train,0,1,0
